# Employee Record Management System

### Objective: Build an employee record management application.

## Requirements:

- Create a system to manage employee records.
- Allow adding, updating, and deleting employee records.
- Implement search functionality based on different criteria (name, department, etc.).
- Display reporting hierarchy of employees.
- Handle edge cases such as circular references in the reporting structure.

In [6]:
class Employee:
    def __init__(self,emp_id,name,department,manager_id = None):
        self.emp_id = emp_id
        self.name = name
        self.department = department
        self.manager_id = manager_id

class EmployeeRecord:
    def __init__(self):
        self.employees = {}

    def add_employee(self,emp_id,name,department,manager_id = None):
        if emp_id in self.employees:
            print(f'Employee with ID {emp_id} already exists.')
            return
        self.employees[emp_id] = Employee(emp_id,name,department,manager_id)
        print(f'Employee {name} added successfully.')

    def update_employee(self,emp_id,name= None,department= None,manager_id = None):
        if emp_id not in self.employees:
            print(f'Employee {emp_id} does not exist.')
            return 
        if name:
            self.employees[emp_id].name = name
        if department:
            self.employees[emp_id].department = department
        if manager_id:
            self.employees[emp_id].manager_id = manager_id
        print(f'Employee {emp_id} detial updated successfully.')

    def delete_employee(self,emp_id):
        if emp_id not in self.employees:
            print(f'Employee with ID {emp_id} does not exits.')
            return 
        del self.employees[emp_id]
        print(f'Employee {emp_id} deleted successfully.')

    def search_employee(self, **kwargs):
        results = []
        for emp in self.employees.values():
            match = True
            for key, value in kwargs.items():
                if getattr(emp,key) != value:
                    match = False
                    break
            if match:
                results.append(emp)
        return results
    
    def display_hi(self,emp_id,visited = None):
        if visited is None:
            visited = set()
        if emp_id not in self.employees:
            print(f'Employee ID {emp_id} does not exits.')
        if emp_id in visited:
            print('Circular reference Detected')
            return
        visited.add(emp_id)
        emp= self.employees[emp_id]
        print(f'Employee ID: {emp.emp_id} Employee Name: {emp.name}, Department: {emp.department}')
        subordinates = [e.emp_id for e in self.employees.values() if e.manager_id == emp_id]
        for sub_id in subordinates:
            self.display_hi(sub_id,visited) 

if __name__ == "__main__":
    emp = EmployeeRecord()

In [7]:
emp.add_employee(1,"Alice", "HR",2)
emp.add_employee(2,"Rahul", "Security",0)
emp.add_employee(3,"Arun","Team Leader",3)
emp.add_employee(4,"Naveem","Production",4)
emp.add_employee(5,"Kumar","Manager",1)
emp.add_employee(6,"Martin","Production",4)

Employee Alice added successfully.
Employee Rahul added successfully.
Employee Arun added successfully.
Employee Naveem added successfully.
Employee Kumar added successfully.
Employee Martin added successfully.


In [8]:
emp.update_employee(4,"Naveem","Senior Production",3)

Employee 4 detial updated successfully.


In [19]:
emp.delete_employee(4)

Employee 4 deleted successfully.


In [20]:
emp.delete_employee(4)

Employee with ID 4 does not exits.


In [4]:
results = emp.search_employee(department="Security")
for emp in results:
    print(f"Found: {emp.name} (ID: {emp.emp_id}, Dept: {emp.department})")

Found: Rahul (ID: 2, Dept: Security)


In [9]:
emp.display_hi(4)

Employee ID: 4 Employee Name: Naveem, Department: Senior Production
Employee ID: 6 Employee Name: Martin, Department: Production


# Call Taxi Booking Application

## Objective: Design a call taxi booking application.

# Requirements:

- There are n taxis, assume 4 for simplicity.
- Six points (A, B, C, D, E, F) in a straight line, 15 kms apart.
- All taxis start at point A.
- It takes 60 minutes to travel from one point to the next.
- When a customer books a taxi, allocate a free taxi at that point. If none are available, allocate a
taxi from the nearest point.
- If two taxis are free at the same point, allocate the one with lower earnings.
- Taxis charge Rs.100 for the first 5 kms and Rs.10 for each subsequent km.
- Taxis only charge from the pickup point to the drop point.
- If no taxi is free at the time, the booking is rejected.

In [12]:
class Taxi:
    def __init__(self,id):
        self.id = id        # Ref Id 
        self.location = 'A' # Start Location
        self.earnings = 0   # Inital Earning
        self.is_free = True # Inital State

class TaxiBookingSystem:
    def __init__(self,points,dis_bw_point,start_location = 'A',taxis = 4):
        self.points = points
        self.dis_bw_point = dis_bw_point # Distance b/w points
        self.time_to_travel = 60 # min to travel
        self.start_location = start_location
        self.taxis = [Taxi(i) for i in range(1, taxis + 1)]

    def BookTaxi(self,pickup_point,drop_point):
        free_taxis = []
        for taxi in self.taxis:
            if taxi.is_free and taxi.location == pickup_point:
                free_taxis.append(taxi)

        if not free_taxis:
            nearest_taxi = self.find_nearest_taxi(pickup_point)
            if not nearest_taxi:
                print('Booking Rejected: No Free Taxi avaiable at the time.')
                return
            taxi = nearest_taxi
            travel_time = self.calculate_travel_time(taxi.location,pickup_point)
            taxi.location = pickup_point
            print(f'Taxi {taxi.id} is assigned from {taxi.location} to {pickup_point}, travel time: {travel_time} minutes')
        else:
            taxi = min(free_taxis,key=lambda x: x.earnings)
        
        fare = self.calculate_fare(pickup_point,drop_point)
        taxi.earnings += fare
        taxi.location = drop_point
        taxi.is_free = False

        print(f'Taxi {taxi.id} booked from {pickup_point} to {drop_point}. Fare: Rs.{fare}')
                  
    def find_nearest_taxi(self,point):
        min_distance = float('inf')
        nearest_taxi = None

        for taxi in self.taxi:
            if taxi.is_free:
                distance = abs(self.points.index(taxi.location)-self.points.index(point))
                if distance < min_distance:
                    min_distance = distance
                    nearest_taxi = taxi
                elif distance == min_distance and taxi.earnings < nearest_taxi.earnings:
                    nearest_taxi = taxi
        return nearest_taxi

    def calculate_travel_time(self,start_point,end_point):
        distance = abs(self.points.index(start_point) - self.points.index(end_point))
        travel_time = distance * self.time_to_travel
        return travel_time

    def calculate_fare(self,pickup_point,drop_point):
        distance = abs(self.points.index(pickup_point) - self.points.index(drop_point)) * self.dis_bw_point
        fare = 200 if distance <= 5 else 100 + (distance - 5 ) * 10
        return fare
    
    def release_taxi(self, taxi_id):
        taxi = next((taxi for taxi in self.taxis if taxi.id == taxi_id), None)
        if taxi:
            taxi.is_free = True
            print(f"Taxi {taxi_id} is now free.")

if __name__ == '__main__':
    points = ['A','B','C','D','E','F']
    distance_between_points = 15
    system = TaxiBookingSystem(points,distance_between_points)

In [13]:
system.BookTaxi('A', 'C')

Taxi 1 booked from A to C. Fare: Rs.350


In [14]:
system.BookTaxi('A', 'B')

Taxi 2 booked from A to B. Fare: Rs.200


In [15]:
system.release_taxi(1)

Taxi 1 is now free.


In [16]:
system.release_taxi(2)

Taxi 2 is now free.


In [17]:
system.BookTaxi('A', 'F')

Taxi 3 booked from A to F. Fare: Rs.800


In [18]:
system.release_taxi(3)

Taxi 3 is now free.
